# Uncertainty in Autonomous Systems
Testing uncertainty in autonomous systems using the Carla simulator.

## Connect to Carla server

In [1]:
import carla
import random
import time

In [2]:
# Connect to the Carla server.
client = carla.Client('localhost', 2000)

## Simulation settings

## Manipulate environment

In [5]:
client.set_timeout(15)
client.load_world('Town02')
client.set_timeout(5)

In [98]:
client.reload_world(reset_settings=True)

In [99]:
# Change simulation settings on the server
world = client.get_world()
settings = world.get_settings()

# Enable synchronous mode to have the server only update on client's ticks.
settings.synchronous_mode = True

# Fixed simulation time step should be used with synchronous mode
settings.substepping = True
settings.max_substep_delta_time = 0.01
settings.max_substeps = 10
settings.fixed_delta_seconds = settings.max_substep_delta_time * settings.max_substeps

world.apply_settings(settings)

5417

In [102]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation
print(location, rotation)

# Set the spectator with an initial transform
spectator.set_transform(carla.Transform(carla.Location(x=95, y=306.5, z=4.3), carla.Rotation(pitch=-15, yaw=2.5)))

Location(x=108.279999, y=307.859985, z=4.310000) Rotation(pitch=0.000000, yaw=-5.801944, roll=0.000000)


In [103]:
world.tick()

5422

## Add actors

In [104]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')

# Show a few examples from the list
print(*list(vehicle_blueprints)[0:5], sep='\n')

ActorBlueprint(id=vehicle.audi.a2,tags=[vehicle, audi, a2])
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
ActorBlueprint(id=vehicle.audi.tt,tags=[vehicle, audi, tt])
ActorBlueprint(id=vehicle.mercedes.coupe_2020,tags=[vehicle, mercedes, coupe_2020])
ActorBlueprint(id=vehicle.bmw.grandtourer,tags=[vehicle, bmw, grandtourer])


In [105]:
# Add the ego vehicle, which will be the centerpoint of the simulation
ego_vehicle_bp = world.get_blueprint_library().find('vehicle.audi.etron')
ego_vehicle_bp.set_attribute('role_name', 'hero')
ego_vehicle = world.spawn_actor(ego_vehicle_bp, carla.Transform(carla.Location(x=100, y=306.5, z=0.5)))

In [108]:
lead_vehicle_bp = world.get_blueprint_library().find('vehicle.mercedes.coupe')
lead_vehicle = world.spawn_actor(lead_vehicle_bp, carla.Transform(carla.Location(x=110, y=306.5, z=0.5)))

In [111]:
world.tick()

5423

In [112]:
print(ego_vehicle.get_location())

Location(x=100.000000, y=306.500000, z=0.446100)


In [110]:
print(lead_vehicle.get_location())

Location(x=0.000000, y=0.000000, z=0.000000)


In [69]:
# # Get the map's spawn points
# spawn_points = world.get_map().get_spawn_points()

# # Spawn 50 vehicles randomly distributed throughout the map 
# # for each spawn point, we choose a random vehicle from the blueprint library
# for i in range(0,50):
#     world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

## Vehicle Control

In [113]:
def cruise_control(ego, lead, reference_distance=10):
    distance = ego.get_location().distance(lead.get_location())
    distance_vec = carla.Vector3D(
        ego.get_location().x - lead.get_location().x,
        ego.get_location().y - lead.get_location().y
    ).make_unit_vector()
    
    rel_velocity = ego.get_velocity() - lead.get_velocity()
    rel_speed = rel_velocity.dot(distance_vec)
    
    acc = 0.05 * (distance - reference_distance)
    throttle = max(acc, 0)
    brake = max(-acc, 0)
    steer = 0
    return throttle, brake, steer

In [118]:
ego_vehicle.set_location(carla.Location(x=100, y=306.5, z=0.5))
lead_vehicle.set_location(carla.Location(x=110, y=306.5, z=0.5))
world.tick()

5623

In [117]:
from IPython.display import display, clear_output

lead_vehicle.set_autopilot(True)
for i in range(1, 100):
    t, b, s = cruise_control(ego_vehicle, lead_vehicle)
    clear_output(wait=True)
    print(t, b, s)
    ego_vehicle.apply_control(carla.VehicleControl(throttle=t, steer=s, brake=b))
    world.tick()
    time.sleep(0.1)
lead_vehicle.set_autopilot(False)

0.17876753807067872 0 0


## Add sensors

In [ ]:
gnss_bp = world.get_blueprint_library().find('sensor.other.gnss')
gnss_bp.set_attribute("sensor_tick",'1.0')

gnss_transform = carla.Transform()
gnss = world.spawn_actor(gnss_bp, gnss_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)

In [199]:
# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1280')
camera_bp.set_attribute('image_size_y', '720')
camera_bp.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
camera_bp.set_attribute('sensor_tick', '1.0')

# Create a transform to place the camera on top of the vehicle
camera_transform = carla.Transform(carla.Location(x=0.8, z=1.5))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego_vehicle)

In [10]:
# We create the camera through a blueprint that defines its properties
depth_camera_bp = world.get_blueprint_library().find('sensor.camera.depth')
depth_camera_bp.set_attribute('image_size_x', '1280')
depth_camera_bp.set_attribute('image_size_y', '720')
depth_camera_bp.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
depth_camera_bp.set_attribute('sensor_tick', '1.0')

# Create a transform to place the camera on top of the vehicle
depth_camera_init_trans = carla.Transform(carla.Location(x=0.8, z=1.5))
depth_camera = world.spawn_actor(depth_camera_bp, depth_camera_init_trans, attach_to=ego_vehicle)

## Animate vehicles


In [11]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

## Collect Data

In [13]:
data_collection = True

In [16]:
data_collection = False

In [12]:
camera = world.get_actors().filter('*sensor.camera.rgb*')[0]
depth_camera = world.get_actors().filter('*sensor.camera.depth*')[0]

In [14]:
camera.listen(lambda image: car_control(distance_estimation(image)))

In [15]:
depth_camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame) if data_collection else 0)

## Manual Control Demo